In [1]:
import pygame
import random
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\astan\anaconda3\envs\py311\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
WIDTH = 300
HEIGHT =300
SCALE = 2

COLOR_EMPTY = (0, 0, 0) 
COLOR_FOOD = (0, 255, 0)
COLOR_OBSTACLE = (50, 50, 50)
COLOR_PARAMECIUM = (255, 0, 0)

SENSE_RADIUS = 10
GENERATE_RADIUS = 20

FOOD_COUNT = 10
OBSTACLE_COUNT = 100
PARAMECIUM_COUNT = 1000

FOOD_RATE = 0
OBSTACLE_RATE = 0

env=[[0 for _ in range(WIDTH)]for _ in range(HEIGHT)]
parameciums_dic = {}
food_click_list = []
add_delay = 0
last_add_time = 0
recursion_num = 0

In [3]:
def check_food_around(env,x,y,radius):
    '''
    x,y : the position of paramecium
    radius : the sense radius of paramecium
    return->the relative position of paramecium or None
    '''
    env[y][x] = 0
    for r in range(1,radius+1):
        for dx in range(-r,r+1):
            for dy in range(-r,r+1):
                if abs(dx) == r or abs(dy) == r:
                    cx = x+dx
                    cy = y+dy
                    if 0<=cx<WIDTH and 0<=cy<HEIGHT :
                        if env[cy][cx] == 1 :
                            env[y][x]=9
                            return dx,dy
    env[y][x] = 9
    return None

In [4]:
def move_paramecium(env,x,y,dx,dy):
    """
    (navigate around obstacles)
    dx,dy : expected x,y offset
    return-> new x,y
    """
    new_x = x + (1 if dx>0 else (-1 if dx<0 else 0))
    new_y = y + (1 if dy>0 else (-1 if dy<0 else 0))

    if new_x < 0 or new_x >= WIDTH or new_y < 0 or new_y >= HEIGHT :
        return x,y
    elif env[new_y][new_x] == 1 or env[new_y][new_x] == 0:
        env[y][x] = 0
        env[new_y][new_x] = 9
        return new_x,new_y
    else :
        return move_randomly(env,x,y)

    # env[y][x] = 0
    # env[new_y][new_x] = 9
    # return new_x,new_y

In [5]:
def draw_environment(env):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            # 根据格子值选择颜色
            if env[y][x] == 1:
                color = COLOR_FOOD
            elif env[y][x] == 2:
                color = COLOR_OBSTACLE
            elif env[y][x] == 9:
                color = COLOR_PARAMECIUM
            else:
                color = COLOR_EMPTY

            pygame.draw.rect(screen,color,(x * SCALE, y * SCALE, SCALE - 1, SCALE - 1))

In [6]:
def move_randomly(env,paramecium_x,paramecium_y):
    """
    It will move randomly when the paramecium is stopping
    stop_random_go() -> (new) paramecium_x,paramecium_y
    """

    global recursion_num
    
    dx = random.choice([-1,0,1])
    dy = random.choice([-1,0,1])
    new_x = paramecium_x + (1 if dx>0 else (-1 if dx<0 else 0))
    new_y = paramecium_y + (1 if dy>0 else (-1 if dy<0 else 0))
    if new_x < 0 or new_x >= WIDTH or new_y < 0 or new_y >= HEIGHT :
        recursion_num += 1
        if recursion_num >= 4 :
            recursion_num = 0
            return paramecium_x,paramecium_y
        new_x,new_y = move_randomly(env,paramecium_x,paramecium_y)
        return new_x,new_y
    elif env[new_y][new_x] == 0 : 
        env[paramecium_y][paramecium_x] = 0
        env[new_y][new_x] = 9
        return new_x,new_y
    else :
        recursion_num += 1
        if recursion_num >= 4 :
            recursion_num = 0
            return paramecium_x,paramecium_y
        new_x,new_y = move_randomly(env,paramecium_x,paramecium_y)
        return new_x,new_y

In [7]:
def generate_plot(rate):
    """
    generate plot randomly,according to the rate(the probability of every frame)
    rate:(for example) 0.2
    generate_plot(rate) -> (new)x,y
    """
    if random.random() < rate :
        x = random.randint(0,WIDTH-1)
        y = random.randint(0,HEIGHT-1)
        return x,y
    else :
        return None

In [8]:
def move_to_food(env,paramecium_x,paramecium_y,SENSE_RADIUS):
    """
    move to the food or move randomly
    SENSE_RADIUS : the radius of the paramecium's sense(if the food is out of the SENSE_RADIUS,it will move randomly)
    move_to_food() -> (new)paramecium_x,paramecium_y
    """
    food_dir = check_food_around(env,paramecium_x,paramecium_y,SENSE_RADIUS)
    if food_dir :
        dx,dy = food_dir
        paramecium_x,paramecium_y=move_paramecium(env,paramecium_x,paramecium_y,dx,dy)
        return paramecium_x,paramecium_y
    else :#randomly move
        paramecium_x,paramecium_y = move_randomly(env,paramecium_x,paramecium_y)
        return paramecium_x,paramecium_y

In [9]:
def move_to_food_tup(env,paramecium_x_y,SENSE_RADIUS):
    """
    move to the food or move randomly
    paramecium_x_y : a tuple of paramecium_x and paramecium_y
    SENSE_RADIUS : the radius of the paramecium's sense(if the food is out of the SENSE_RADIUS,it will move randomly)
    move_to_food() -> (new)paramecium_x,paramecium_y
    """
    paramecium_x,paramecium_y = paramecium_x_y
    food_dir = check_food_around(env,paramecium_x,paramecium_y,SENSE_RADIUS)
    if food_dir :
        dx,dy = food_dir
        paramecium_x,paramecium_y=move_paramecium(env,paramecium_x,paramecium_y,dx,dy)
        return paramecium_x,paramecium_y
    else :#randomly move
        paramecium_x,paramecium_y = move_randomly(env,paramecium_x,paramecium_y)
        return paramecium_x,paramecium_y

In [10]:
def generate_paramecium_randomly(env):
    """
    generate the paramecium randomly
    generate_paramecium_randomly() -> (new)paramecium_x,paramecium_y
    """
    paramecium_x,paramecium_y = generate_plot(1)
    env[paramecium_y][paramecium_x] = 9
    return paramecium_x,paramecium_y

In [11]:
def generate_num_paramecium(env,num):
    """
    generate the num paramecium randomly and put it into the parameciums_dic
    parameciums_dic : (for example) {'parameciuma_0': (12, 4), 'parameciuma_1': (2, 3)}
    generate_num_paramecium() -> None
    """
    for i in range(num) :
        paramecium_x,paramecium_y=generate_paramecium_randomly(env)
        parameciums_dic[f'parameciuma_{i}'] = (paramecium_x,paramecium_y)

In [12]:
#initialize the environment
pygame.init()
screen = pygame.display.set_mode((WIDTH*SCALE,HEIGHT*SCALE))
pygame.display.set_caption("simulate paramecium")
clock = pygame.time.Clock()

#initialize the paramecium
# paramecium_1_x,paramecium_1_y=generate_paramecium_randomly(env)
# paramecium_2_x,paramecium_2_y=generate_paramecium_randomly(env)
for paramecium_num in range(1,PARAMECIUM_COUNT+1):
    parameciums_dic[paramecium_num] = generate_paramecium_randomly(env)

#create the food randomly
for _ in range(FOOD_COUNT):
    while True :
        x=random.randint(0,WIDTH-1)
        y=random.randint(0,HEIGHT-1)
        if env[y][x]==0 :
            env[y][x]==1
            break
            
#create the barrier
for _ in range(OBSTACLE_COUNT):
    while True :
        x=random.randint(0,WIDTH-1)
        y=random.randint(0,HEIGHT-1)
        if env[y][x]==0 :
            env[y][x]=2
            break

In [13]:
running = True
pygame.key.set_repeat(200,50)
while running :
    current_time = pygame.time.get_ticks()
    clock.tick(30)
    keys = pygame.key.get_pressed()
    for event in pygame.event.get() :
        if event.type == pygame.QUIT :
            running = False
        # if event.type == pygame.MOUSEBUTTONDOWN :
        #     click_x,click_y = event.pos
        #     food_click_list.append((click_x,click_y))
    if keys[pygame.K_ESCAPE] :
        running = False

    left_pressed = pygame.mouse.get_pressed()[0]
    right_pressed = pygame.mouse.get_pressed()[2]
    if left_pressed and current_time - last_add_time > add_delay :
        press_x,press_y = pygame.mouse.get_pos()
        food_click_list.append((press_x,press_y))
        last_add_time = current_time
    if right_pressed : 
        press_x,press_y = pygame.mouse.get_pos()
        for dx in range(-GENERATE_RADIUS,GENERATE_RADIUS+1):
            for dy in range(-GENERATE_RADIUS,GENERATE_RADIUS+1):
                new_x = press_x//SCALE + dx
                new_y = press_y//SCALE + dy
                if new_y >= 0 and new_x < WIDTH and new_x >= 0 and new_y < HEIGHT :
                    if env[new_y][new_x] != 9 :  
                        env[new_y][new_x] = 1
    
    # paramecium_1_x,paramecium_1_y = move_to_food(env,paramecium_1_x,paramecium_1_y,SENSE_RADIUS)
    # paramecium_2_x,paramecium_2_y = move_to_food(env,paramecium_2_x,paramecium_2_y,SENSE_RADIUS)
    for paramecium_num in parameciums_dic.keys():
        parameciums_dic[paramecium_num] = move_to_food_tup(env,parameciums_dic[paramecium_num],SENSE_RADIUS)

    #generate food plot
    x_y = generate_plot(FOOD_RATE)
    if x_y :
        x,y = x_y
        if env[y][x] == 0 :
            env[y][x] = 1
            
    if food_click_list :
        for click_x_y in food_click_list:
            click_x,click_y = click_x_y
            click_x = click_x//SCALE
            click_y = click_y//SCALE
            # print(click_x,click_y)
            if click_y >= 0 and click_x < WIDTH and click_x >= 0 and click_y < HEIGHT :
                if env[click_y][click_x] == 0 or env[click_y][click_x] == 2:
                    env[click_y][click_x] = 1
    food_click_list.clear()

    #generate obstacle plot            
    # while True :
    #     x_y = generate_plot(OBSTACLE_RATE)
    #     if x_y :
    #         x,y = x_y
    #         if env[y][x] == 0 :
    #             env[y][x] = 2
    #             break
    x_y = generate_plot(OBSTACLE_RATE)
    if x_y :
        x,y = x_y
        if env[y][x] == 0 :
            env[y][x] = 2

    # screen.fill(COLOR_EMPTY)
    draw_environment(env)
    pygame.display.update()

pygame.quit()
sys.exit()

SystemExit: 

C:\Users\astan\anaconda3\envs\py311\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
